In [ ]:
!pip install transformers
!pip install datasets
!pip install wandb
!pip install torch

In [ ]:
import transformers




In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
df=pd.read_csv('social_media_sentiment_dataset.csv')
df.head()

In [ ]:
df_new=df.drop('ID',axis=1)
df_new

In [ ]:
df_new.isnull().sum()

In [ ]:
df_new["Sentiemt"]=df_new.rename(columns={"Sentiment":"labels"},inplace=True)

In [ ]:
df_new.head()

In [ ]:
df_new.drop('Sentiemt',axis=1,inplace=True)

In [ ]:
df_new.head(1)

In [ ]:
label_encoder=LabelEncoder()
df_new['labels']=label_encoder.fit_transform(df_new['labels'])
df_new.head(3)

In [ ]:
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

In [ ]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


In [ ]:
# Function to tokenize each comment
def tokenize(example):
    return tokenizer(example["Comment"], truncation=True, padding="max_length", max_length=128)


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_new[["Comment", "labels"]])
tokenized_dataset = dataset.map(tokenize)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_dataset, test_dataset = tokenized_dataset.train_test_split(test_size=0.2).values()

In [ ]:
print(f"Training data size: {len(train_dataset)}")
print(f"Test data size: {len(test_dataset)}")

In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


In [ ]:
!pip install transformers[torch]
!pip install "accelerate>=0.26.0"

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    #evaluation_strategy="epoch",             # must evaluate per epoch for early stopping
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",                   # must save to compare best mode            # important!
    metric_for_best_model="eval_loss",       # or use "accuracy" if you're tracking that
    greater_is_better=False,                 # since lower loss is better
)


In [ ]:
model = BartForSequenceClassification.from_pretrained("facebook/bart-base", num_labels=len(df_new["labels"].unique()))

In [ ]:
trainer = Trainer(
    model=model,                      # The model to train
    args=training_args,               # The training arguments
    train_dataset=train_dataset,      # The training dataset
    eval_dataset=test_dataset,        # The evaluation dataset
    tokenizer=tokenizer,               # The tokenizer for preprocessing the data

)


In [ ]:
import wandb
wandb.init(project="bart-sentiment-classification")


In [ ]:
# Training the model
trainer.train()


In [ ]:
import numpy as np

def predic(new_data):
  new_dataset = Dataset.from_dict({"Comment": new_data})
  tokenized_new = new_dataset.map(tokenize)
  predictions = trainer.predict(tokenized_new)
  pred_logits = predictions.predictions[0]
  predicted_labels = np.argmax(pred_logits, axis=1)
  return predicted_labels



In [ ]:
print("Enter the Sentence")
var=input()

decoded_predictions = label_encoder.inverse_transform(predic([var]))
print(decoded_predictions)
